In [11]:
from roboflow import Roboflow
import os
from PIL import Image, ImageDraw, ImageFont


In [12]:
# Trying different pre trained models


### Comment and uncomment different models to test them.
### Put your own API keys below and run the code

rf = Roboflow(api_key="put your api key")
# project = rf.workspace().project("wooden-pallets-detection")
# model = project.version(5).model

# project = rf.workspace().project("pallets-0mhou")
# model = project.version(2).model

# project = rf.workspace().project("deepstream-model")
# model = project.version(1).model

project = rf.workspace().project("pallet-detector-2a8o6")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [13]:
# all paths

IMAGE_PATH = 'images'
PREDICTION_PATH = 'prediction'
MARKED_POINTS_PATH = 'marked_points'
IMAGE_EXT = ['.png', '.jpg', '.jpeg']

In [14]:
def infer():
    metadata = {}
    for file in os.listdir(IMAGE_PATH):
        if file.lower().endswith(tuple(IMAGE_EXT)):
            image = os.path.join(IMAGE_PATH, file)
            pred = PREDICTION_PATH + '/' + file
            # visualize your prediction
            model.predict(image, confidence=20, overlap=30).save(pred)
            prediction = model.predict(image, confidence=20, overlap=30).json()
            if prediction:
                coords = [[each['x'], each['y'], each['width'], each['height'], each['confidence']*100] for each in prediction.get('predictions', [])]
                metadata[image] = coords
    return metadata

# infer()

In [15]:
# # infer on a local image
print(model.predict("images/image16.jpeg", confidence=20, overlap=30).json())

{'predictions': [{'x': 201.5, 'y': 158.0, 'width': 85.0, 'height': 22.0, 'confidence': 0.8456639647483826, 'class': 'Wooden-Pallets', 'class_id': 1, 'image_path': 'images/image16.jpeg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 84.0, 'y': 160.5, 'width': 124.0, 'height': 23.0, 'confidence': 0.8224821090698242, 'class': 'Wooden-Pallets', 'class_id': 1, 'image_path': 'images/image16.jpeg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '269', 'height': '188'}}


In [33]:
def add_text_tags(image, coordinates_and_tags):
#     # Open the image
#     image = Image.open(image_path)

    # Create a drawing object
    draw = ImageDraw.Draw(image)

    # Font settings (adjust as needed)
    font_size = 10
    font_color = "yellow"
    # Use a default font (no font path specified)
    font = ImageFont.load_default()

    # Add text tags near given coordinates
    x, y, tag = coordinates_and_tags
    # Calculate position for the text
    text_x = x - 15  # Adjust as needed
    text_y = y - 25  # Adjust as needed

    # Add text to the image
    draw.text((text_x, text_y), tag, font=font, fill=font_color)
    
    return image

#     # Save the image with text tags
#     image.save(output_path)


# # Example usage:
# image_path = 'path/to/your/image.jpg'
# output_path = 'path/to/save/image_with_text_tags.jpg'
# coordinates_and_tags = [(100, 150, "Tag1"), (200, 250, "Tag2"), (300, 100, "Tag3")]

# add_text_tags(image_path, coordinates_and_tags, output_path)


In [35]:
def mark_coordinates(metadata):
    for image_path, coordinates in metadata.items():
#         import pdb;pdb.set_trace()
        output_path = MARKED_POINTS_PATH + "/" + image_path.split('/')[-1]
        # Open the image
        image = Image.open(image_path)

        # Create a drawing object
        draw = ImageDraw.Draw(image)

        # Define the size of the marker (adjust as needed)
        marker_size = 10

        # Loop through the coordinates and draw markers
        for x, y, w, h, conf in coordinates:
            # Calculate the coordinates for outer box
            left = x - w // 2
            top = y - h // 2
            right = x + w // 2
            bottom = y + h // 2
            # Draw a rectangle to mark the coordinates
            draw.rectangle([left, top, right, bottom], outline="yellow", width=1)
            coord_tag = [x, y, f'{x},{y}\nconf: {conf:.2f}%']
            image = add_text_tags(image, coord_tag)
            draw = ImageDraw.Draw(image)


            # Calculate the coordinates for left inner box 
            left = (x - w // 2) + 1
            top = (y - h // 2) + 1
            right = x - 1
            bottom = (y + h // 2) - 1
            # Draw a rectangle to mark the coordinates
            draw.rectangle([left, top, right, bottom], outline="green", width=1)

            # Calculate the center of the left inner box
            center_x = (left + right) // 2
            center_y = (top + bottom) // 2

            # Highlight the center with a marker 
            marker_size = 5
            marker_left = center_x - marker_size // 2
            marker_top = center_y - marker_size // 2
            marker_right = center_x + marker_size // 2
            marker_bottom = center_y + marker_size // 2
            draw.rectangle([marker_left, marker_top, marker_right, marker_bottom], outline="green", width=2)


            # Calculate the coordinates for right inner box 
            left = x + 1
            top = (y - h // 2) + 1
            right = (x + w // 2) - 1
            bottom = (y + h // 2) - 1
            # Draw a rectangle to mark the coordinates
            draw.rectangle([left, top, right, bottom], outline="green", width=1)

            # Calculate the center of the right inner box
            center_x = (left + right) // 2
            center_y = (top + bottom) // 2

            # Highlight the center with a marker 
            marker_size = 5
            marker_left = center_x - marker_size // 2
            marker_top = center_y - marker_size // 2
            marker_right = center_x + marker_size // 2
            marker_bottom = center_y + marker_size // 2
            draw.rectangle([marker_left, marker_top, marker_right, marker_bottom], outline="green", width=2)



        # Save the marked image
        rgb_im = image.convert('RGB')
        rgb_im.save(output_path)


# mark_coordinates()


In [18]:
metadata = infer()
# print(metadata)

In [36]:
mark_coordinates(metadata)

In [32]:
# # !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="2AG2ctUU8KxikJI5kWf1")
# project = rf.workspace("arapl").project("pallet-detector-2a8o6")
# dataset = project.version(1).download("yolov5")
